In [ ]:
## Part One Feature Engineering

In [ ]:

import os, gc, random
import numpy as np, pandas as pd
from numpy.fft import rfft, rfftfreq

GOOD_CSV  = "/kaggle/input/nasa-bearing-dataset/train_final.csv"   # raw healthy data
FAULT_CSV = "/kaggle/working/merged_test.csv"  # raw faulty data
OUT_DIR   = "/kaggle/working/features"; os.makedirs(OUT_DIR, exist_ok=True)

WIN  = 2048
HOP  = 512
FFT_N = 1024
EPS  = 1e-9


def robust_norm(x):
    med = np.median(x); mad = np.median(np.abs(x - med)) + EPS
    z = (x - med) / (1.4826 * mad)
    return np.clip(z, -6.0, 6.0)

def time_features(x):
    mean = float(np.mean(x))
    std  = float(np.std(x) + EPS)
    rms  = float(np.sqrt(np.mean(x*x)))
    mad  = float(np.mean(np.abs(x - mean)))
    p2p  = float(np.max(x) - np.min(x))
    skew = float(np.mean(((x-mean)/std)**3))
    kurt = float(np.mean(((x-mean)/std)**4))
    crest= float(np.max(np.abs(x)) / (rms + EPS))
    imp  = float((np.max(np.abs(x)) + EPS) / (np.mean(np.abs(x)) + EPS))
    return [mean,std,rms,mad,p2p,skew,kurt,crest,imp]

def freq_features(x, fs=1.0, nfft=FFT_N):
    if len(x) < nfft:
        pad = np.zeros(nfft, dtype=np.float32); pad[:len(x)] = x
        x = pad
    X = np.abs(rfft(x))
    X = X / (np.sum(X) + EPS)
    freqs = rfftfreq(len(x), d=1.0/fs)
    centroid = float(np.sum(freqs * X))
    bandwidth = float(np.sqrt(np.sum(((freqs-centroid)**2)*X)))
    csum = np.cumsum(X); idx = np.searchsorted(csum, 0.95)
    rolloff = float(freqs[min(idx, len(freqs)-1)])
    geo = np.exp(np.mean(np.log(X + EPS))); arith = np.mean(X) + EPS
    flatness = float(geo/arith)
    entropy = float(-np.sum(X * np.log(X + EPS)))
    dom = float(freqs[np.argmax(X)])
    top2 = np.partition(X, -2)[-2:]
    peak_ratio = float((top2[-1]+EPS)/(top2[-2]+EPS))
    q = len(X)//4
    bands = [float(np.sum(X[i*q:(i+1)*q])) for i in range(4)]
    return [centroid, bandwidth, rolloff, flatness, entropy, dom, peak_ratio] + bands

def window_iter(buf):
    n = buf.shape[0]
    for s in range(0, n - WIN + 1, HOP):
        yield buf[s:s+WIN]

def detect_bearing_cols(df):
    cols = [c for c in df.columns if c.lower().startswith("bearing")]
    if len(cols) < 4:
        raise ValueError(f"Expected ≥4 bearing columns, got {cols}")
    return cols[:4]

def process_csv(path, tag):
    feats = []
    prev_tail = None
    for df in pd.read_csv(path, chunksize=2_000_000):
        cols = detect_bearing_cols(df)
        X = df[cols].astype(np.float32).values
        if prev_tail is not None:
            X = np.vstack([prev_tail, X])
        for win in window_iter(X):
            if np.all(np.var(win, axis=0) < 1e-8): 
                continue
            fvec = []
            for ch in range(win.shape[1]):
                x = robust_norm(win[:, ch])
                fvec += time_features(x) + freq_features(x)
            feats.append(fvec)
        prev_tail = X[-(WIN-HOP):].copy()
        del df, X; gc.collect()
    df_out = pd.DataFrame(feats)
    out_path = os.path.join(OUT_DIR, f"features_{tag}.csv")
    df_out.to_csv(out_path, index=False)
    print(f"Saved {len(df_out)} windows to {out_path}")

if __name__ == "__main__":
    process_csv(GOOD_CSV, "good")
    process_csv(FAULT_CSV, "fault")


In [ ]:
## Part two, training
ENABLED = {"AE": False, "SEN": False, "VAE": True}
SAVE_WEIGHTS = {"AE": False, "SEN": False, "VAE": True}
EXPORT_VAE = True


import os
SEED_MASTER = 13
os.environ["PYTHONHASHSEED"] = str(SEED_MASTER)
os.environ["TF_DETERMINISTIC_OPS"] = "1"
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"


import json, random, gc, joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from mpl_toolkits.mplot3d import Axes3D  #
import matplotlib.pyplot as plt
from sklearn.metrics import (
    roc_auc_score, average_precision_score,
    roc_curve, precision_recall_curve, confusion_matrix
)
from sklearn.manifold import trustworthiness
from sklearn.decomposition import PCA
from sklearn.preprocessing import RobustScaler

FEAT_GOOD  = "./features/features_good.csv"
FEAT_FAULT = "./features/features_fault.csv"
OUT_DIR    = "./outputs_toggle"; os.makedirs(OUT_DIR, exist_ok=True)


SEEDS       = [303]  ## pore lagle aro add hobe eikahne


EPOCHS_AE   = 30
EPOCHS_SEN  = 35
EPOCHS_VAE  = 40
BATCH       = 256
HID         = [256, 128]
LATENT      = 8

warm-up
BETA_START, BETA_END, WARMUP_EPOCHS = 0.0, 1.0, 20


SEN_NOISE_STD = 0.02  # training-time only


N_NEIGH     = 10
USE_FLOAT16 = True
MC_RUNS     = 5
VIEW_ANGLES_3D = [
    dict(elev=90, azim=200,  name="viewA"),
    dict(elev=30, azim=180,  name="viewB"),
]


OUTPUT_LINEAR = False 


tf.config.threading.set_intra_op_parallelism_threads(1)
tf.config.threading.set_inter_op_parallelism_threads(1)
try:
    tf.keras.utils.set_random_seed(SEED_MASTER)
except Exception:
    random.seed(SEED_MASTER); np.random.seed(SEED_MASTER); tf.random.set_seed(SEED_MASTER)


def set_seed(s: int):
    random.seed(s); np.random.seed(s); tf.random.set_seed(s)

def get_optimizer(lr=1e-3):
    if hasattr(tf.keras.optimizers, "AdamW"):
        return tf.keras.optimizers.AdamW(learning_rate=lr)
    return tf.keras.optimizers.Adam(learning_rate=lr)


def load_and_scale(noise_std=0.005, use_robust=True):
    Xg = pd.read_csv(FEAT_GOOD).values.astype(np.float32)   # healthy
    Xf = pd.read_csv(FEAT_FAULT).values.astype(np.float32)  # faulty

    if use_robust:
        scaler = RobustScaler()
        Xg = scaler.fit_transform(Xg)
        Xf = scaler.transform(Xf)
        scaler_info = scaler
    else:
        mn = np.percentile(Xg, 1, axis=0)
        mx = np.percentile(Xg, 99, axis=0)
        Xg = np.clip((Xg - mn)/(mx - mn + 1e-6), 0, 1)
        Xf = np.clip((Xf - mn)/(mx - mn + 1e-6), 0, 1)
        scaler_info = dict(mn=mn, mx=mx)

    rng = np.random.default_rng(42)
    Xg = Xg + rng.normal(0, noise_std, Xg.shape).astype(np.float32)
    if not use_robust:
        Xg = np.clip(Xg, 0, 1)

    if USE_FLOAT16:
        Xg, Xf = Xg.astype(np.float16), Xf.astype(np.float16)
    return Xg, Xf, scaler_info

def make_ds(X, batch, shuffle=False, seed=None):
    ds = tf.data.Dataset.from_tensor_slices((X, X))
    if shuffle:
        ds = ds.shuffle(buffer_size=min(10000, len(X)),
                        seed=seed, reshuffle_each_iteration=False)
    opts = tf.data.Options()
    opts.experimental_deterministic = True
    ds = ds.with_options(opts)
    return ds.batch(batch, drop_remainder=False).prefetch(1)

def recon_errors(model, X, bs=1024, deterministic=True):
    errs = []
    for i in range(0, len(X), bs):
        xb = X[i:i+bs].astype(np.float32)
        if hasattr(model, "encode_mu") and deterministic:
            mu = model.encode_mu(xb)
            xh = model.decode(mu).numpy()
        else:
            xh = model.predict(xb, verbose=0)
            if hasattr(xh, "numpy"):
                xh = xh.numpy()
        errs.append(np.mean((xh - xb)**2, axis=1))
    return np.concatenate(errs, 0)

def sample_for_manifold(X, max_n=10000, seed=123):
    if len(X) <= max_n: return X
    rng = np.random.default_rng(seed)
    return X[rng.choice(len(X), size=max_n, replace=False)]

def thr_percentile(errs, p=99.5): return float(np.percentile(errs, p))

def thr_evt(errs, tail_p=5.0, q=1e-3):
    u = np.percentile(errs, 100 - tail_p)
    y = errs[errs >= u] - u
    if len(y) < 200:
        return thr_percentile(errs, 99.5)
    y = np.sort(y); k = len(y)
    x1, x2, x4 = y[int(0.25*k)], y[int(0.5*k)], y[int(0.75*k)]
    xi = (np.log((x4 - x2) + 1e-9) - np.log((x2 - x1) + 1e-9)) / np.log(2)
    xi = np.clip(xi, -0.49, 0.49)
    beta = (1 + xi) * np.mean(y)
    thr = u + (beta/xi) * ((q**(-xi)) - 1) if abs(xi) > 1e-6 else u - beta*np.log(q)
    return float(thr)


def plot_hist(err_g, err_f, tag, pmax=99.5, use_log=False):
 
    import numpy as np, matplotlib.pyplot as plt, os
    eps = 1e-12
    err_g = np.asarray(err_g, dtype=np.float64); err_g = err_g[np.isfinite(err_g)]
    err_f = np.asarray(err_f, dtype=np.float64); err_f = err_f[np.isfinite(err_f)]
    if use_log:
        xg = np.log10(np.maximum(err_g, eps)); xf = np.log10(np.maximum(err_f, eps))
        xlabel = "log10(MSE)"
    else:
        xg, xf = err_g, err_f
        xlabel = "MSE"
    lo = min(xg.min(initial=0.0), xf.min(initial=0.0))
    hi = np.percentile(np.concatenate([xg, xf]), pmax)
    if not np.isfinite(hi) or hi <= lo:
        hi = max(xg.max(initial=1.0), xf.max(initial=1.0))
    bins = np.linspace(lo, hi, 100)

    plt.figure()
    plt.hist(xg, bins=bins, density=True, alpha=0.6, label="Without Fault")
    plt.hist(xf, bins=bins, density=True, alpha=0.6, label="Near Faulty")
    plt.legend(); plt.xlabel(xlabel); plt.ylabel("pdf")
    plt.title(tag + (" (robust)" if not use_log else " (log-scale)"))
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, f"{tag}.png"), dpi=150)
    plt.close()
    print(f"[{tag}] {xlabel} range used: [{lo:.4g}, {hi:.4g}] "
          f"| medians: H={np.median(xg):.4g}, F={np.median(xf):.4g}")

# -----------------------------------------------------------
def plot_roc_pr(y_true, scores, tag):
    fpr, tpr, _ = roc_curve(y_true, scores)
    prec, rec, _ = precision_recall_curve(y_true, scores)
    plt.figure(); plt.plot(fpr, tpr); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title(f"ROC {tag}")
    plt.savefig(os.path.join(OUT_DIR, f"roc_{tag}.png"), dpi=150); plt.close()
    plt.figure(); plt.plot(rec, prec); plt.xlabel("Recall"); plt.ylabel("Precision"); plt.title(f"PR {tag}")
    plt.savefig(os.path.join(OUT_DIR, f"pr_{tag}.png"), dpi=150); plt.close()

def _stabilize_pca_signs(pca: PCA):
    comps = pca.components_.copy()
    for i in range(comps.shape[0]):
        j = np.argmax(np.abs(comps[i]))
        if comps[i, j] < 0:
            comps[i] *= -1.0
    pca.components_ = comps
    return pca

def _apply_pca_projection(mu, mean, comps):
    return (mu - mean) @ comps.T


def save_latent_pca3_npz(mu_g, mu_f, out_npz_path):
    pca3 = PCA(n_components=3, svd_solver="full").fit(mu_g)
    comps = pca3.components_.copy()
    for i in range(comps.shape[0]):
        j = np.argmax(np.abs(comps[i]))
        if comps[i, j] < 0: comps[i] *= -1.0
    mean_ref3 = pca3.mean_.astype(np.float32)
    comps_ref3 = comps.astype(np.float32)
    Zg3 = _apply_pca_projection(mu_g, mean_ref3, comps_ref3).astype(np.float32)
    Zf3 = _apply_pca_projection(mu_f, mean_ref3, comps_ref3).astype(np.float32)
    np.savez(out_npz_path,
             Zg3=Zg3, Zf3=Zf3,
             mean=mean_ref3, components=comps_ref3,
             evr=pca3.explained_variance_ratio_.astype(np.float32))

def plot_latent(mu_g, mu_f, tag):
    # 2D PCA
    pca2 = PCA(n_components=2, svd_solver="full")
    Zg2 = pca2.fit_transform(mu_g)
    pca2 = _stabilize_pca_signs(pca2)
    mean_ref2 = pca2.mean_.copy(); comps_ref2 = pca2.components_.copy()
    Zg2 = _apply_pca_projection(mu_g, mean_ref2, comps_ref2)
    Zf2 = _apply_pca_projection(mu_f, mean_ref2, comps_ref2)
    evr2 = pca2.explained_variance_ratio_.copy()
    plt.figure()
    plt.scatter(Zg2[:,0], Zg2[:,1], s=4, alpha=0.5, label="healthy")
    plt.scatter(Zf2[:,0], Zf2[:,1], s=4, alpha=0.6, label="faulty")
    plt.legend(); plt.title(f"Latent PCA μ(z) – 2D ({tag})")
    plt.xlabel("PC1"); plt.ylabel("PC2")
    plt.savefig(os.path.join(OUT_DIR, f"latent_pca_2d_{tag}.png"), dpi=150); plt.close()
    
    # 3D views pCA
    pca3 = PCA(n_components=3, svd_solver="full")
    Zg3 = pca3.fit_transform(mu_g)
    pca3 = _stabilize_pca_signs(pca3)
    mean_ref3 = pca3.mean_.copy(); comps_ref3 = pca3.components_.copy()
    Zg3 = _apply_pca_projection(mu_g, mean_ref3, comps_ref3)
    Zf3 = _apply_pca_projection(mu_f, mean_ref3, comps_ref3)
    for v in VIEW_ANGLES_3D:
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.scatter(Zg3[:,0], Zg3[:,1], Zg3[:,2], s=3, alpha=0.3, label="healthy")
        ax.scatter(Zf3[:,0], Zf3[:,1], Zf3[:,2], s=9, alpha=0.85, label="faulty")
        ax.set_xlabel("PC1"); ax.set_ylabel("PC2"); ax.set_zlabel("PC3")
        ax.legend(); ax.set_title(f"Latent PCA μ(z) – 3D ({tag})")
        ax.view_init(elev=v["elev"], azim=v["azim"])
        plt.savefig(os.path.join(OUT_DIR, f"latent_pca_3d_{tag}_{v['name']}.png"), dpi=150)
        plt.close()
    return evr2

def dense_param_count(a, b): return a*b + b
def dense_macs(a, b): return a*b

def ae_like_stats(inp, hidden, latent):
    sizes = [inp] + hidden + [latent]
    sizes_dec = [latent] + list(reversed(hidden)) + [inp]
    p = sum(dense_param_count(a,b) for a,b in zip(sizes[:-1], sizes[1:]))
    p += sum(dense_param_count(a,b) for a,b in zip(sizes_dec[:-1], sizes_dec[1:]))
    m = sum(dense_macs(a,b) for a,b in zip(sizes[:-1], sizes[1:]))
    m += sum(dense_macs(a,b) for a,b in zip(sizes_dec[:-1], sizes_dec[1:]))
    return p, m

def vae_stats(inp, hidden, latent):
    sizes_enc = [inp] + hidden
    sizes_dec = [latent] + list(reversed(hidden)) + [inp]
    p = sum(dense_param_count(a,b) for a,b in zip(sizes_enc[:-1], sizes_enc[1:]))
    p += dense_param_count(hidden[-1], latent) * 2 
    p += sum(dense_param_count(a,b) for a,b in zip(sizes_dec[:-1], sizes_dec[1:]))
    m = sum(dense_macs(a,b) for a,b in zip(sizes_enc[:-1], sizes_enc[1:]))
    m += dense_macs(hidden[-1], latent) * 2
    m += sum(dense_macs(a,b) for a,b in zip(sizes_dec[:-1], sizes_dec[1:]))
    return p, m

def estimate_latency_ms(macs, mmacs_per_s=50.0):
    return (macs / (mmacs_per_s * 1e6)) * 1000.0

def pretty_stats(name, input_dim, hidden, latent, keras_model=None, tflite_path=None):
    lname = name.lower()
    if lname == "vae":
        P, M = vae_stats(input_dim, hidden, latent)
    else:
        P, M = ae_like_stats(input_dim, hidden, latent)
    kp = None
    if keras_model is not None:
        try: _ = keras_model(tf.zeros((1, input_dim), dtype=tf.float32))
        except Exception: pass
        try: kp = keras_model.count_params()
        except: kp = None
    fs = None
    if tflite_path and os.path.exists(tflite_path):
        fs = os.path.getsize(tflite_path) / 1024.0
    print(f"\n[{name}] Params≈{P:,}" + (f" (Keras={kp:,})" if kp is not None else " (Keras=N/A)"))
    print(f"  MACs≈{M:,} (~{M/1e9:.4f} GMACs), FLOPs≈{2*M/1e9:.4f} GFLOPs")
    if fs: print(f"  TFLite size≈{fs:.1f} KiB")
    print(f"  ~Latency≈{estimate_latency_ms(M, 50.0):.4f} ms @ 50 MMAC/s")

class AE(tf.keras.Model):
    def __init__(self, input_dim, hidden, latent):
        super().__init__(name="AE")
        self.enc_layers = [tf.keras.layers.Dense(h, activation="relu") for h in hidden]
        self.mu = tf.keras.layers.Dense(latent, name="bottleneck")
        self.dec_layers = [tf.keras.layers.Dense(h, activation="relu") for h in reversed(hidden)]
        act = None if OUTPUT_LINEAR else "sigmoid"
        self.out_layer  = tf.keras.layers.Dense(input_dim, activation=act)
        self.mse = tf.keras.losses.MeanSquaredError()
    def encode(self, x):
        h = x
        for l in self.enc_layers: h = l(h)
        return self.mu(h)
    def encode_mu(self, x):
        x = tf.convert_to_tensor(x, dtype=tf.float32)
        return self.encode(x)
    def decode(self, z):
        h = z
        for l in self.dec_layers: h = l(h)
        return self.out_layer(h)
    def call(self, x, training=False):
        return self.decode(self.encode(x))

def build_ae(input_dim):
    ae = AE(input_dim, HID, LATENT)
    ae.compile(optimizer=get_optimizer(1e-3), loss="mse")
    return ae

class SEN(tf.keras.Model):
    def __init__(self, input_dim, hidden, latent, noise_std=0.02):
        super().__init__(name="SEN")
        self.noise_std = noise_std
        self.enc_layers = [tf.keras.layers.Dense(h, activation="relu") for h in hidden]
        self.mu = tf.keras.layers.Dense(latent)
        self.dec_layers = [tf.keras.layers.Dense(h, activation="relu") for h in reversed(hidden)]
        act = None if OUTPUT_LINEAR else "sigmoid"
        self.out_layer  = tf.keras.layers.Dense(input_dim, activation=act)
        self.mse = tf.keras.losses.MeanSquaredError()
    def encode(self, x, training=False):
        h = x
        for l in self.enc_layers: h = l(h)
        mu = self.mu(h)
        z = mu + tf.random.normal(tf.shape(mu), stddev=self.noise_std) if training and self.noise_std>0 else mu
        return mu, z
    def decode(self, z):
        h = z
        for l in self.dec_layers: h = l(h)
        return self.out_layer(h)
    def call(self, x, training=False):
        _, z = self.encode(x, training=training)
        return self.decode(z)
    def train_step(self, data):
        x = data[0] if isinstance(data, tuple) else data
        with tf.GradientTape() as tape:
            mu, z = self.encode(x, training=True)
            xh = self.decode(z)
            loss = self.mse(x, xh)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return {"loss": loss}
    def test_step(self, data):
        x = data[0] if isinstance(data, tuple) else data
        mu, z = self.encode(x, training=False)
        xh = self.decode(z)
        loss = self.mse(x, xh)
        return {"loss": loss}
    def encode_mu(self, x):
        x = tf.convert_to_tensor(x, dtype=tf.float32)
        h = x
        for l in self.enc_layers: h = l(h)
        return self.mu(h)

def build_sen(input_dim, noise_std=SEN_NOISE_STD):
    sen = SEN(input_dim, HID, LATENT, noise_std=noise_std)
    sen.compile(optimizer=get_optimizer(1e-3))
    return sen

class VAE(tf.keras.Model):
    def __init__(self, input_dim, hidden, latent, beta=1.0, noise_std=0.01):
        super().__init__(name="VAE")
        self.beta = beta
        self.noise_std = noise_std
        self.enc_layers = [tf.keras.layers.Dense(h, activation="relu") for h in hidden]
        self.mu      = tf.keras.layers.Dense(latent)
        self.logvar  = tf.keras.layers.Dense(latent)
        self.dec_layers = [tf.keras.layers.Dense(h, activation="relu") for h in reversed(hidden)]
        act = None if OUTPUT_LINEAR else "sigmoid"
        self.out_layer  = tf.keras.layers.Dense(input_dim, activation=act)
        self.mse = tf.keras.losses.MeanSquaredError()
    def encode(self, x, training=False):
        if training and self.noise_std>0: x = x + tf.random.normal(tf.shape(x), stddev=self.noise_std)
        h = x
        for l in self.enc_layers: h = l(h)
        return self.mu(h), self.logvar(h)
    def decode(self, z):
        h = z
        for l in self.dec_layers: h = l(h)
        return self.out_layer(h)
    def sample(self, mu, logvar):
        return mu + tf.exp(0.5 * logvar) * tf.random.normal(tf.shape(mu))
    def call(self, x, training=False):
        mu, logvar = self.encode(x, training=training)
        z  = self.sample(mu, logvar)
        return self.decode(z)
    def train_step(self, data):
        x = data[0] if isinstance(data, tuple) else data
        with tf.GradientTape() as tape:
            mu, logvar = self.encode(x, training=True)
            z  = self.sample(mu, logvar)
            xh = self.decode(z)
            recon = self.mse(x, xh)
            kl = -0.5 * tf.reduce_mean(1 + logvar - tf.square(mu) - tf.exp(logvar))
            loss = recon + self.beta * kl
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        return {"loss": loss, "recon": recon, "kl": kl}
    def test_step(self, data):
        x = data[0] if isinstance(data, tuple) else data
        mu, logvar = self.encode(x, training=False)
        z  = self.sample(mu, logvar)
        xh = self.decode(z)
        recon = self.mse(x, xh)
        kl = -0.5 * tf.reduce_mean(1 + logvar - tf.square(mu) - tf.exp(logvar))
        return {"loss": recon + self.beta * kl, "recon": recon, "kl": kl}
    def encode_mu(self, x):
        x = tf.convert_to_tensor(x, dtype=tf.float32)
        mu, _ = self.encode(x, training=False)
        return mu

def build_vae(input_dim, beta=1.0, noise_std=0.01):
    vae = VAE(input_dim, HID, LATENT, beta=beta, noise_std=noise_std)
    vae.compile(optimizer=get_optimizer(1e-3))
    return vae

class BetaWarmup(tf.keras.callbacks.Callback):
    def __init__(self, model, beta_start, beta_end, warmup_epochs):
        super().__init__(); self.m = model
        self.b0 = beta_start; self.b1 = beta_end; self.w = warmup_epochs
    def on_epoch_begin(self, epoch, logs=None):
        t = min(1.0, (epoch + 1) / float(self.w))
        self.m.beta = self.b0 + t * (self.b1 - self.b0)

def build_export_infer(vae_model, input_dim):
    xin = tf.keras.Input(shape=(input_dim,), name="input_features")
    h   = xin
    for layer in vae_model.enc_layers: h = layer(h)
    mu  = vae_model.mu(h)
    xout = vae_model.decode(mu)
    return tf.keras.Model(xin, xout, name="vae_mu_decoder_infer")


def evaluate_model(model_name, plot_tag, model, Xte_g, Xf, Xg_all_sampled, latent_fn):
    err_g = recon_errors(model, Xte_g, deterministic=True)
    err_f = recon_errors(model, Xf,    deterministic=True)

    # thresholds
    thr_p = thr_percentile(err_g)
    thr_e = thr_evt(err_g)

    y_true = np.concatenate([np.zeros_like(err_g), np.ones_like(err_f)])
    scores = np.concatenate([err_g, err_f])
    auroc  = roc_auc_score(y_true, scores)
    aupr   = average_precision_score(y_true, scores)

    plot_hist(err_g, err_f, f"{plot_tag}_hist", pmax=99.5, use_log=False)
    plot_hist(err_g, err_f, f"{plot_tag}_hist_log", pmax=99.9, use_log=True)

    plot_roc_pr(y_true, scores, f"{plot_tag}")

    uncert = None
    if model_name in ("SEN", "VAE"):
        Ss = [recon_errors(model, Xf, deterministic=False) for _ in range(MC_RUNS)]
        uncert = float(np.mean(np.std(Ss, 0)))

    mu_g = latent_fn(Xg_all_sampled.astype(np.float32)).numpy()
    mu_f = latent_fn(sample_for_manifold(Xf).astype(np.float32)).numpy()
    tw   = float(trustworthiness(Xg_all_sampled, mu_g, n_neighbors=N_NEIGH))
    cont = float(trustworthiness(mu_g, Xg_all_sampled, n_neighbors=N_NEIGH))
    evr  = plot_latent(mu_g, mu_f, plot_tag)

    npz_path = os.path.join(OUT_DIR, f"latent_pca3_{plot_tag}.npz")
    save_latent_pca3_npz(mu_g, mu_f, npz_path)

    cm = confusion_matrix(y_true, (scores >= thr_p).astype(int))
    tn, fp, fn, tp = cm.ravel()
    prec  = tp / (tp + fp + 1e-9)
    recall= tp / (tp + fn + 1e-9)
    f1    = 2 * prec * recall / (prec + recall + 1e-9)

    return dict(
        model=model_name, auroc=float(auroc), aupr=float(aupr),
        thr_percentile=thr_p, thr_evt=thr_e,
        trustworthiness=tw, continuity=cont,
        uncert_fault=uncert,
        precision=float(prec), recall=float(recall), f1=float(f1),
        cm=cm.tolist(),
        pca_explained_variance_ratio=evr.tolist(),
        pca_npz=npz_path
    )

if __name__ == "__main__":
    Xg, Xf, scaler_info = load_and_scale(noise_std=0.005, use_robust=True)

    n = len(Xg)
    i_tr, i_va = int(0.6 * n), int(0.8 * n)
    Xtr, Xva, Xte_g = Xg[:i_tr], Xg[i_tr:i_va], Xg[i_va:]
    input_dim = Xtr.shape[1]
    Xg_s = sample_for_manifold(Xte_g)
    all_seed_results = []

    for s in SEEDS:
        print(f"\n================= SEED {s} =================")
        set_seed(s)
        train_ds = make_ds(Xtr.astype(np.float32), BATCH, True, seed=s)
        val_ds   = make_ds(Xva.astype(np.float32),  BATCH, False, seed=s)
        results_this_seed = []

        if ENABLED.get("AE", False):
            ae = build_ae(input_dim)
            _ = ae(tf.zeros((1, input_dim), dtype=tf.float32))
            pretty_stats("AE", input_dim, HID, LATENT, keras_model=ae)
            ae.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_AE, verbose=2,
                   callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)])
            res_ae = evaluate_model("AE", f"AE_seed{s}", ae, Xte_g, Xf, Xg_s,
                                    latent_fn=lambda X: ae.encode_mu(X))
            results_this_seed.append(res_ae)
            if SAVE_WEIGHTS.get("AE", False):
                ae.save_weights(os.path.join(OUT_DIR, f"ae_seed{s}.weights.h5"))
            del ae; tf.keras.backend.clear_session(); gc.collect(); set_seed(s)


        if ENABLED.get("SEN", False):
            sen = build_sen(input_dim, noise_std=SEN_NOISE_STD)
            _ = sen(tf.zeros((1, input_dim), dtype=tf.float32))
            pretty_stats("SEN", input_dim, HID, LATENT, keras_model=sen)
            sen.fit(train_ds, validation_data=val_ds, epochs=EPOCHS_SEN, verbose=2,
                    callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)])
            res_sen = evaluate_model("SEN", f"SEN_seed{s}", sen, Xte_g, Xf, Xg_s,
                                     latent_fn=lambda X: sen.encode_mu(X))
            results_this_seed.append(res_sen)
            if SAVE_WEIGHTS.get("SEN", False):
                sen.save_weights(os.path.join(OUT_DIR, f"sen_seed{s}.weights.h5"))
            del sen; tf.keras.backend.clear_session(); gc.collect(); set_seed(s)

        if ENABLED.get("VAE", True):
            vae = build_vae(input_dim, beta=BETA_END, noise_std=0.01)
            _ = vae(tf.zeros((1, input_dim), dtype=tf.float32))
            pretty_stats("VAE", input_dim, HID, LATENT, keras_model=vae)
            vae.fit(
                train_ds, validation_data=val_ds, epochs=EPOCHS_VAE, verbose=2,
                callbacks=[
                    BetaWarmup(vae, BETA_START, BETA_END, WARMUP_EPOCHS),
                    tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True)
                ]
            )
            res_vae = evaluate_model("VAE", f"VAE_seed{s}", vae, Xte_g, Xf, Xg_s,
                                     latent_fn=lambda X: vae.encode_mu(X))
            results_this_seed.append(res_vae)
            if SAVE_WEIGHTS.get("VAE", True):
                vae.save_weights(os.path.join(OUT_DIR, f"vae_seed{s}.weights.h5"))
            del vae; tf.keras.backend.clear_session(); gc.collect(); set_seed(s)

        all_seed_results.append(dict(seed=s, results=results_this_seed))

    rows = []
    for item in all_seed_results:
        for r in item["results"]:
            rows.append(dict(seed=item["seed"], **r))

    if len(rows) == 0:
        print("No models enabled; nothing to aggregate.")
        raise SystemExit(0)

    df = pd.DataFrame(rows)
    df.to_csv(os.path.join(OUT_DIR, "results_all_seeds.csv"), index=False)

    agg = df.groupby("model").agg(
        auroc_mean=("auroc","mean"), auroc_std=("auroc","std"),
        aupr_mean=("aupr","mean"), aupr_std=("aupr","std"),
        f1_mean=("f1","mean"), f1_std=("f1","std"),
        precision_mean=("precision","mean"), precision_std=("precision","std"),
        recall_mean=("recall","mean"), recall_std=("recall","std"),
        trustworthiness_mean=("trustworthiness","mean"), trustworthiness_std=("trustworthiness","std"),
        continuity_mean=("continuity","mean"), continuity_std=("continuity","std"),
        uncert_fault_mean=("uncert_fault","mean"), uncert_fault_std=("uncert_fault","std")
    ).reset_index()
    agg.to_csv(os.path.join(OUT_DIR, "results_summary.csv"), index=False)

    with open(os.path.join(OUT_DIR, "results_all.json"), "w") as f:
        json.dump(all_seed_results, f, indent=2)
    with open(os.path.join(OUT_DIR, "results_summary.json"), "w") as f:
        json.dump(agg.to_dict(orient="records"), f, indent=2)

    if ENABLED.get("VAE", True) and EXPORT_VAE:
        df_vae = df[df.model=="VAE"]
        if len(df_vae) == 0:
            print("VAE not present in results; skipping export.")
        else:
            best_seed = df_vae.sort_values("auroc", ascending=False).iloc[0]["seed"]
            print("Best seed for VAE export:", best_seed)

            set_seed(int(best_seed))
            Xtr, Xva = Xtr.astype(np.float32), Xva.astype(np.float32)
            Xtr_full = np.vstack([Xtr, Xva])
            train_full_ds = make_ds(Xtr_full, BATCH, True, seed=int(best_seed))

            vae_final = build_vae(input_dim, beta=BETA_END, noise_std=0.01)
            _ = vae_final(tf.zeros((1, input_dim), dtype=tf.float32))
            vae_final.fit(
                train_full_ds, epochs=EPOCHS_VAE, verbose=2,
                callbacks=[
                    BetaWarmup(vae_final, BETA_START, BETA_END, WARMUP_EPOCHS),
                    tf.keras.callbacks.EarlyStopping(monitor="loss", patience=6, restore_best_weights=True)
                ]
            )

            if isinstance(scaler_info, dict):
                np.savez(os.path.join(OUT_DIR, "scaler_min_max.npz"), **scaler_info)
            else:
                joblib.dump(scaler_info, os.path.join(OUT_DIR, "robust_scaler.pkl"))

            infer = build_export_infer(vae_final, input_dim)
            infer.save(os.path.join(OUT_DIR, "vae_fp32.keras"))

            def rep_ds():
                Xrep = Xtr_full
                for i in range(0, min(20000, len(Xrep)), 256):
                    yield [Xrep[i:i+256]]

            conv = tf.lite.TFLiteConverter.from_keras_model(infer)
            conv.optimizations = [tf.lite.Optimize.DEFAULT]
            conv.representative_dataset = rep_ds
            conv.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
            conv.inference_input_type  = tf.int8
            conv.inference_output_type = tf.int8
            tfl = conv.convert()
            tfl_path = os.path.join(OUT_DIR, "vae_int8.tflite")
            with open(tfl_path, "wb") as f:
                f.write(tfl)

            best_row = df_vae[df_vae.seed==best_seed].iloc[0]
            with open(os.path.join(OUT_DIR, "deployment_thresholds.json"), "w") as f:
                json.dump({
                    "best_seed": int(best_seed),
                    "thr_percentile": float(best_row["thr_percentile"]),
                    "thr_evt": float(best_row["thr_evt"])
                }, f, indent=2)

            pretty_stats("VAE final INT8", input_dim, HID, LATENT, tflite_path=tfl_path)
            print("Exported model + scaler + thresholds to", OUT_DIR)
    else:
        print("VAE export disabled or VAE not enabled; skipping export.")
